In [1]:
from typing import cast

import sqil_core as sqil
from sqil_core.config_log import logger

# Connect to the server and retrieve instruments
server, instruments = sqil.experiment.link_instrument_server()

# Check available instruments
logger.info("Connected to the instruments server")
logger.info(f" instruments: {list(instruments.keys())}")

# Do something
# sgs = cast(sqil.experiment.LocalOscillator, instruments["sgs"])
# logger.debug("Requesting SGS frequency change to 11 GHz")
# sgs.frequency(11e9)
zi_session = instruments["zi"]


# Disconnect from the instrument server and release variables
sqil.experiment.unlink_instrument_server(server, **instruments)

484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81 484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81


2025-04-28 20:19:19,385 - INFO - Connected to the instruments server
2025-04-28 20:19:19,386 - INFO -  instruments: ['sgs', 'zi']


In [ ]:
import sqil_core as sqil

class Test(sqil.experiment.ExperimentHandler):
    def sequence(self):
        print("Hello")

t = Test()
t.run()

484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81 484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81
[2025.04.28 20:21:30.434] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is c:\Users\sqil\Desktop\code\sqil-experiments\measurements\laboneq_output\log


2025-04-28 20:21:30,435 - INFO - Turning on local oscillators before experiment


Hello


2025-04-28 20:21:30,436 - INFO - Turning off local oscillators after experiment


In [ ]:
import sqil_core as sqil
import numpy as np
from rr_spec import create_experiment
from laboneq.dsl.quantum import QPU
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonOperations,
    TunableTransmonQubit,
)

class Test(sqil.experiment.ExperimentHandler):

    def sequence(self):
        qubits = TunableTransmonQubit.from_device_setup(self.zi_setup)
        quantum_operations = TunableTransmonOperations()
        qpu = QPU(qubits=qubits, quantum_operations=quantum_operations)

        qpu.qubits[0].update(
            **{
                "drive_lo_frequency": 5e9,
                "readout_lo_frequency": 7.2e9,
                "readout_resonator_frequency": 7.4e9,
            }
        )

        return create_experiment(qpu, qubits[0], np.linspace(7.3e9, 8e9, 101))

t = Test()
result = t.run()

484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81 484e0fbec4043279f7eef813c84b5f55b7939b3a8bf01e3a59824e6ea1055a81
[2025.05.01 11:59:52.435] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is c:\Users\sqil\Desktop\code\sqil-experiments\measurements\laboneq_output\log
[2025.05.01 11:59:52.444] INFO    VERSION: laboneq 2.50.0
[2025.05.01 11:59:52.446] INFO    Connecting to data server at localhost:8004
[2025.05.01 11:59:52.453] INFO    Connected to Zurich Instruments LabOne Data Server version 25.04.0.628 at localhost:8004
[2025.05.01 11:59:52.454] CRITICAL Version of LabOne Data Server (25.04.0.628) and Python API (25.01.0.834) do not match.


[2025.05.01 11:59:52.454] CRITICAL Version of LabOne Data Server (25.04.0.628) and Python API (25.01.0.834) do not match.


LabOneQControllerException: Version of LabOne Data Server (25.04.0.628) and Python API (25.01.0.834) do not match.

In [37]:
result

{
│   'errors': [],
│   'q0': {
│   │   'result': AcquiredResult(
│   │   │   data=array([ 2.78608549e-04+1.33689928e-04j, -6.45304719e-05-2.85227970e-04j,
│      -8.25259633e-05+2.23293784e-04j,  2.78938811e-04-1.44685199e-04j,
│      -2.41203148e-04-1.03075372e-04j,  1.62908366e-04+1.72388085e-04j,
│   │   4.90629988e-05-2.86213190e-04j, -1.95085945e-04+1.13221933e-04j,
│   │   2.40423654e-04-1.71499285e-05j, -1.23378583e-04-2.10670216e-04j,
│      -2.69307015e-05+1.55984714e-04j,  1.55020814e-04-1.84545764e-04j,
│      -1.83289401e-04-1.00043414e-04j,  6.39549659e-05+5.26190347e-05j,
│   │   1.17718363e-05-2.32798817e-04j, -2.47667464e-04-4.50440353e-05j,
│   │   6.65869490e-05+1.10357500e-05j, -1.71597518e-04-2.20562149e-04j,
│      -2.41030483e-04+1.62811815e-04j,  7.51507230e-05-9.61466216e-06j,
│      -3.35038439e-04-1.07009009e-04j, -1.06072713e-04+3.80714450e-04j,
│   │   4.36842457e-05-9.14267463e-05j, -4.34152128e-04+2.48747263e-04j,
│   │   2.90520506e-04+4.28424092e-04j, -2.81152579e-05-1.86050378e-04j,
│      -1.48472409e-04+6.16287509e-04j,  6.73050347e-04+1.22290178e-05j,
│      -3.48982022e-04-1.29628847e-04j,  5.03801821e-04+6.32983499e-04j,
│   │   4.07587409e-04-6.85337839e-04j, -3.96897018e-04+3.66209660e-04j,
│   │   1.00115429e-03-2.82054573e-05j, -4.54052300e-04-8.39986303e-04j,
│   │   9.02010453e-05+7.55839324e-04j,  6.46281370e-04-1.02320790e-03j,
│      -1.22539141e-03+4.70263107e-05j,  9.51632789e-04+5.21497562e-04j,
│      -5.74786201e-04-1.38805620e-03j, -7.91394797e-04+1.45608258e-03j,
│   │   1.61432299e-03-7.05010679e-04j, -2.26112770e-03-6.37291507e-04j,
│   │   1.51361634e-03+2.66353819e-03j,  7.55271821e-04-4.24934319e-03j,
│      -6.14997581e-03+5.75589190e-03j,  2.63927605e-02-3.19439776e-02j,
│   │   4.49903525e-03+9.15879195e-03j,  8.64736385e-04-4.95513924e-03j,
│      -2.53778084e-03+2.09821916e-03j,  2.41759923e-03-7.47572044e-05j,
│      -1.60695312e-03-1.30596303e-03j,  1.64031346e-04+1.62108788e-03j,
│   │   5.80512073e-04-1.14494163e-03j, -1.21040896e-03+4.69809073e-04j,
│   │   8.78644875e-04+4.82631525e-04j, -4.47206082e-04-6.09164560e-04j,
│      -9.28818864e-05+9.31153385e-04j,  6.59033418e-04-2.67113255e-04j,
│      -4.12858520e-04+9.39065866e-05j,  6.29342192e-04+3.63084436e-04j,
│   │   1.88286410e-04-5.22603451e-04j, -6.14377886e-05+1.30410673e-04j,
│   │   4.79653461e-04-2.75805095e-04j, -2.33757609e-04-4.33933231e-04j,
│   │   3.00717637e-05+1.59918182e-05j, -2.84679940e-05-4.59779806e-04j,
│      -4.55698141e-04-7.93562824e-05j, -7.96242970e-05+4.48344751e-05j,
│      -3.10856572e-04-1.42405082e-04j, -3.02063977e-04+3.16105049e-04j,
│   │   1.23957286e-05+9.31623664e-05j, -2.85511623e-04+1.18266506e-04j,
│   │   5.33753149e-07+3.92839144e-04j,  7.58476779e-05+7.60548621e-05j,
│      -6.49434162e-05+3.21341251e-04j,  3.67538366e-04+2.87983745e-04j,
│   │   1.57198429e-04-3.09293640e-06j,  2.76109197e-04+1.98252628e-04j,
│   │   3.85344982e-04-1.69092694e-04j,  6.21695007e-05-1.16304620e-04j,
│   │   3.13903138e-04-8.16631827e-05j,  7.21952851e-05-4.12476714e-04j,
│      -1.95687489e-05-1.34318040e-04j,  7.15446541e-05-3.31701388e-04j,
│      -2.64549087e-04-2.65729151e-04j, -1.10931782e-04-8.37020936e-05j,
│      -2.57762820e-04-2.52341212e-04j, -3.16433037e-04+1.04375099e-04j,
│      -1.37084971e-04+4.79180482e-06j, -3.26421995e-04+1.34094678e-04j,
│      -7.48255364e-05+2.74934878e-04j, -5.68957954e-05+1.25885165e-04j,
│      -1.24838229e-05+3.04225062e-04j,  1.94546868e-04+1.42139697e-04j,
│   │   8.74444777e-05+8.80880389e-05j,  2.01671574e-04+8.87805381e-05j,
│   │   1.59631202e-04-8.06149622e-05j,  8.84200828e-05-2.56063622e-05j,
│   │   1.09916298e-04-1.02250031e-04j,  5.99228005e-07-9.17847444e-05j,
│   │   1.91479158e-05-6.13474775e-05j]),
│   │   │   axis_name=[
│   │   │   │   'frequencies_q0'
│   │   │   ],
│   │   │   axis=[
│   │   │   │   array([7.300e+09, 7.307e+09, 7.314e+09, 7.321e+09, 7.328e+09, 7.335e+09,
│      7.342e+09, 7.349e+09, 7.356e+09, 7.363

In [29]:
from laboneq import serializers
from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
from laboneq.dsl.quantum import QPU
from laboneq.simple import DeviceSetup
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonOperations,
    TunableTransmonQubit,
)
from laboneq.dsl.session import Session

from helpers.laboneq import print_qpu_signals

from laboneq_applications.experiments import resonator_spectroscopy
from laboneq_applications.experiments.options import TuneUpWorkflowOptions
import matplotlib.pyplot as plt

# Zurich instruments stetup
zi_descriptor = generate_descriptor(
    shfqc_6=["dev12183"],
    number_data_qubits=1,
    number_flux_lines=0,
    include_cr_lines=False,
    multiplex=True,
    number_multiplex=1,
    get_zsync=False,
    ip_address="localhost",
)

zi_setup = DeviceSetup.from_descriptor(zi_descriptor, "localhost")
zi_session = Session(zi_setup)
zi_session.connect()

qubits = TunableTransmonQubit.from_device_setup(zi_setup)
quantum_operations = TunableTransmonOperations()
qpu = QPU(qubits=qubits, quantum_operations=quantum_operations)

qpu.qubits[0].update(**{"drive_lo_frequency": 5e9})
qpu.qubits[0].update(**{"readout_lo_frequency": 7.2e9})
qpu.qubits[0].update(**{"readout_resonator_frequency": 7.4e9})

experiment_workflow = resonator_spectroscopy.experiment_workflow(
    session=zi_session,
    qpu=qpu,
    qubit=qubits[0],
    frequencies=np.linspace(7.3e9, 8e9, 101),
    options={"do_analysis":True}
)

workflow_result = experiment_workflow.run()
plt.show()

[2025.04.29 14:54:22.787] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is c:\Users\sqil\Desktop\code\sqil-experiments\measurements\laboneq_output\log
[2025.04.29 14:54:22.787] INFO    VERSION: laboneq 2.50.0
[2025.04.29 14:54:22.787] INFO    Connecting to data server at localhost:8004
[2025.04.29 14:54:22.798] INFO    Connected to Zurich Instruments LabOne Data Server version 25.01.0.834 at localhost:8004
[2025.04.29 14:54:22.850] WARNING SHFQC/QA:dev12183: Include the device options 'SHFQC/QC4CH' in the device setup ('options' field of the 'instruments' list in the device setup descriptor, 'device_options' argument when constructing instrument objects to be added to 'DeviceSetup' instances). This will become a strict requirement in the future.
[2025.04.29 14:54:22.855] INFO    Configuring the device setup
[2025.04.29 14:54:22.907] INFO    The device setup is configured
[2025.04.29 14:54:22.915] INFO     ────────────────────────────────────

In [32]:
analysis_workflow_results = workflow_result.tasks["analysis_workflow"]
analysis_workflow_results.output

{'old_parameter_values': {'q0': {'readout_resonator_frequency': 7400000000.0}},
 'new_parameter_values': {'q0': {'readout_resonator_frequency': 7748000000.0+/-0}}}